# 实体记忆
本笔记本展示如何使用记忆模块来记住特定实体的信息。它使用LLM提取实体信息，并随着时间的推移建立其关于该实体的知识（也使用LLM）。

让我们首先通过使用此功能来走一遍流程。

In [1]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

In [2]:
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [3]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [4]:
memory = ConversationEntityMemory(llm=llm, return_messages=True)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

In [5]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': [HumanMessage(content='Deven & Sam are working on a hackathon project', additional_kwargs={}),
  AIMessage(content=' That sounds like a great project! What kind of project are they working on?', additional_kwargs={})],
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

## 在链中使用
现在让我们在链式中使用它！

In [6]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [7]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [8]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great project! What kind of project are they working on?'

In [14]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam, which they are entering into a hackathon.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [15]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like an interesting project! What kind of memory structures are they trying to add?'

In [16]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great idea! How will the key-value store help with the project?'

In [17]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Deven and Sam are working on a hackathon project together, trying to add more complex memory structures to Langchain, including a key-value store for entities mentioned so far in the conversation. They seem to be working hard on this project and have a great idea for how the key-value store can help.'

## 检查记忆库
我们还可以直接检查记忆库。在以下示例中，我们直接查看它，然后通过一些添加信息的示例来观察它是如何发生变化的。

In [21]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Daimon is a company founded by Sam, a successful entrepreneur.',
 'Deven': 'Deven is working on a hackathon project with Sam, which they are '
          'entering into a hackathon. They are trying to add more complex '
          'memory structures to Langchain, including a key-value store for '
          'entities mentioned so far in the conversation, and seem to be '
          'working hard on this project with a great idea for how the '
          'key-value store can help.',
 'Key-Value Store': 'A key-value store is being added to the project to store '
                    'entities mentioned in the conversation.',
 'Langchain': 'Langchain is a project that is trying to add more complex '
              'memory structures, including a key-value store for entities '
              'mentioned so far in the conversation.',
 'Sam': 'Sam is working on a hackathon project with Deven, trying to add more '
        'complex memory structures to Langchain, including a key-value store

In [22]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

" That's impressive! It sounds like Sam is a very successful entrepreneur. What kind of company is Daimon?"

In [24]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Daimon is a company founded by Sam, a successful entrepreneur, who '
           'is working on a hackathon project with Deven to add more complex '
           'memory structures to Langchain.',
 'Deven': 'Deven is working on a hackathon project with Sam, which they are '
          'entering into a hackathon. They are trying to add more complex '
          'memory structures to Langchain, including a key-value store for '
          'entities mentioned so far in the conversation, and seem to be '
          'working hard on this project with a great idea for how the '
          'key-value store can help.',
 'Key-Value Store': 'A key-value store is being added to the project to store '
                    'entities mentioned in the conversation.',
 'Langchain': 'Langchain is a project that is trying to add more complex '
              'memory structures, including a key-value store for entities '
              'mentioned so far in the conversation.',
 'Sam': 'Sam is working on 

In [25]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Sam is the founder of a successful company called Daimon. He is also working on a hackathon project with Deven to add more complex memory structures to Langchain. They seem to have a great idea for how the key-value store can help.'